Optimal Charging and Routing of Electric Vehicles (EVs)

Objective:

Routing Optimization: Determine the most efficient routes for EVs from origin to destination considering factors like distance, traffic, and road conditions.

Charging Optimization: Identify optimal charging stations along the route to minimize charging time and costs while ensuring EVs do not deplete their batteries.

***

Key Features:

Integration of real-time or historical traffic data.

Battery level monitoring and prediction.

Charging station availability and pricing optimization.

User preferences (e.g., fastest route vs. cheapest charging options).

In [ ]:
# Dataset:  https://data.open-power-system-data.org/renewable_power_plants/2020-08-25

In [ ]:
!pip install osmnx

In [ ]:
!pip install pulp

In [ ]:
import osmnx as ox
import requests
import pandas as pd
from pandas import json_normalize
from geopy.geocoders import Nominatim
import networkx as nx
import folium
# Example using PuLP for linear optimization
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary


## Data Collection and Preprocessing

a. Retrieve EV Charging Stations Data

In [ ]:
# EV Charging Stations Dataset (Open Charge Map):

# https://openchargemap.org/site/develop/api

# Road Network and Routing Data (OpenStreetMap):

# https://www.openstreetmap.org/

# Traffic Data:

# HERE Traffic API: https://developer.here.com/products/traffic

# OpenTraffic: https://opentraffic.io/

# EV Specifications (EPA Vehicle Data):

# https://www.fueleconomy.gov/feg/ws/index.shtml

# ------------------------------------------------------------------------------

# Example using Open Charge Map API
API_KEY = 'API'  # Replace with your API key
url = 'https://api.openchargemap.io/v3/poi/'

params = {
    'output': 'json',
    'countrycode': 'US',  # Specify country code as needed
    'maxresults': 1000,
    'compact': True,
    'verbose': False,
    'key': API_KEY
}

response = requests.get(url, params=params)
charging_data = response.json()


In [ ]:
charging_data

[{'IsRecentlyVerified': True,
  'DateLastVerified': '2024-09-03T15:39:00Z',
  'ID': 303710,
  'UUID': 'F456E474-7EBB-4E63-833C-682F84E2558C',
  'DataProviderID': 1,
  'OperatorID': 5,
  'UsageTypeID': 5,
  'UsageCost': '$0.50/kWh',
  'AddressInfo': {'ID': 304099,
   'Title': 'Melloy Dodge',
   'AddressLine1': '9621 Coors Blvd',
   'Town': 'Albuquerque',
   'StateOrProvince': 'New Mexico',
   'Postcode': '87114',
   'CountryID': 2,
   'Latitude': 35.18991807180117,
   'Longitude': -106.63638615493508,
   'AccessComments': 'South entrance',
   'DistanceUnit': 0},
  'Connections': [{'ID': 573214,
    'ConnectionTypeID': 1,
    'StatusTypeID': 50,
    'LevelID': 2,
    'Amps': 80,
    'Voltage': 208,
    'PowerKW': 19.2,
    'CurrentTypeID': 10,
    'Quantity': 2,
    'Comments': 'Near Building'},
   {'ID': 573215,
    'ConnectionTypeID': 32,
    'StatusTypeID': 50,
    'LevelID': 3,
    'Amps': 200,
    'Voltage': 480,
    'PowerKW': 125,
    'CurrentTypeID': 30,
    'Quantity': 2,
    'C

In [ ]:
extracted_data = []

for entry in charging_data:
    if 'AddressInfo' in entry:
        latitude = entry['AddressInfo'].get('Latitude', None)
        longitude = entry['AddressInfo'].get('Longitude', None)
        title = entry['AddressInfo'].get('Title', None)
        for connection in entry.get('Connections', []):
            extracted_data.append({
                'Latitude': latitude,
                'Longitude': longitude,
                'Title': title,
                # Add other fields from the connection if needed
            })

charging_df1 = pd.DataFrame(extracted_data)
print(charging_df1.head())

    Latitude   Longitude                        Title
0  35.189918 -106.636386                 Melloy Dodge
1  35.189918 -106.636386                 Melloy Dodge
2  35.189918 -106.636386                 Melloy Dodge
3  35.189918 -106.636386                 Melloy Dodge
4  43.038698 -108.379586  Fremont Chevrolet Buick GMC


In [ ]:
len(charging_df1)

1246

In [ ]:
# Flatten the JSON data with the available fields
charging_df2 = json_normalize(charging_data,
                             record_path=['Connections'],
                             meta=['AddressInfo.Latitude',
                                   'AddressInfo.Longitude',
                                   'AddressInfo.Title'],
                             errors='ignore')

# Select the available columns
charging_df2 = charging_df2[['AddressInfo.Latitude', 'AddressInfo.Longitude', 'AddressInfo.Title',
                           'ConnectionTypeID', 'Amps', 'Voltage', 'PowerKW', 'Quantity', 'Comments']]

# Display the first few rows to verify the selection
print(charging_df2.head())

  AddressInfo.Latitude AddressInfo.Longitude AddressInfo.Title  \
0                  NaN                   NaN               NaN   
1                  NaN                   NaN               NaN   
2                  NaN                   NaN               NaN   
3                  NaN                   NaN               NaN   
4                  NaN                   NaN               NaN   

   ConnectionTypeID   Amps  Voltage  PowerKW  Quantity            Comments  
0                 1   80.0    208.0     19.2         2       Near Building  
1                32  200.0    480.0    125.0         2  Under solar canopy  
2                 2  200.0    480.0    125.0         2  Under solar canopy  
3                32  400.0    480.0    180.0         2  Under solar canopy  
4                32    NaN      NaN     60.0         2                 NaN  


In [ ]:
charging_df2 = charging_df2.drop(charging_df2.columns[:3], axis=1)

In [ ]:
print(charging_df2.head())

   ConnectionTypeID   Amps  Voltage  PowerKW  Quantity            Comments
0                 1   80.0    208.0     19.2         2       Near Building
1                32  200.0    480.0    125.0         2  Under solar canopy
2                 2  200.0    480.0    125.0         2  Under solar canopy
3                32  400.0    480.0    180.0         2  Under solar canopy
4                32    NaN      NaN     60.0         2                 NaN


In [ ]:
len(charging_df2)

1246

In [ ]:
charging_df = pd.concat([charging_df1, charging_df2], axis=1)

In [ ]:
charging_df.shape

(1246, 9)

In [ ]:
charging_df.columns = ['Latitude', 'Longitude', 'Station_Name', 'ConnectionTypeID', 'Amps', 'Voltage', 'PowerKW', 'Quantity', 'Comments']

# Display the first few rows to verify
print(charging_df.head())


    Latitude   Longitude                 Station_Name  ConnectionTypeID  \
0  35.189918 -106.636386                 Melloy Dodge                 1   
1  35.189918 -106.636386                 Melloy Dodge                32   
2  35.189918 -106.636386                 Melloy Dodge                 2   
3  35.189918 -106.636386                 Melloy Dodge                32   
4  43.038698 -108.379586  Fremont Chevrolet Buick GMC                32   

    Amps  Voltage  PowerKW  Quantity            Comments  
0   80.0    208.0     19.2         2       Near Building  
1  200.0    480.0    125.0         2  Under solar canopy  
2  200.0    480.0    125.0         2  Under solar canopy  
3  400.0    480.0    180.0         2  Under solar canopy  
4    NaN      NaN     60.0         2                 NaN  


b. Download and Prepare Road Network Data

In [ ]:
# Define the area of interest
place = "San Francisco, California, USA"

# Download the street network
G = ox.graph_from_place(place, network_type='drive')

# Attempt to simplify the graph and handle the error
try:
    G = ox.simplify_graph(G)
except:
    print("Graph has already been simplified. Skipping simplification.")
    G.simplified = True

# Get node and edge data
nodes, edges = ox.graph_to_gdfs(G)

Graph has already been simplified. Skipping simplification.


c. Obtain Traffic Data

Note: Traffic data integration depends on the chosen API. Here's a conceptual example.

In [ ]:
# API ID: beGP5pHvdK7Xip6dNpjI
# API KEY: uGmZJmrJp8qyasBBu9QA_P9pJJqMwchevPw8wUiHRas

In [ ]:
# Example using HERE Traffic API
TRAFFIC_API_KEY = 'API' # Replace with your API key
traffic_url = 'https://traffic.ls.hereapi.com/traffic/6.3/flow.json'

params = {
    'apiKey': TRAFFIC_API_KEY,
    'bbox': '37.7749,-122.4194;37.8044,-122.2712',  # Define bounding box
}

response = requests.get(traffic_url, params=params)
traffic_data = response.json()


In [ ]:
traffic_data

{'title': 'Forbidden.',
 'status': 403,
 'code': '403403',
 'cause': 'App beGP5pHvdK7Xip6dNpjI credentials do not authorize access to perform GET action on hrn:here:tdaflowinc::HERE:traffic_63_flow via SERVICE-3839cf60-7483-43a1-98c9-3d150a4d9af0 because no matching permissions found for the identity, its groups and roles, or the realm.',
 'action': 'Add/Share the necessary permissions to the identity.',
 'correlationId': '36260000-f51e-464f-9dd3-d654536f5b9d',
 'error': 'Forbidden',
 'details': [],
 'error_description': 'These credentials do not authorize access'}

## Routing Optimization

a. Define Origin and Destination

In [ ]:
geolocator = Nominatim(user_agent="ev_routing")
origin = geolocator.geocode("Golden Gate Bridge, San Francisco, CA")
destination = geolocator.geocode("Fisherman's Wharf, San Francisco, CA")

origin_point = (origin.latitude, origin.longitude)
destination_point = (destination.latitude, destination.longitude)

In [ ]:
destination_point

(37.8081325, -122.4165913)

b. Find Nearest Nodes in the Graph

In [ ]:
origin_node = ox.distance.nearest_nodes(G, origin.longitude, origin.latitude)
destination_node = ox.distance.nearest_nodes(G, destination.longitude, destination.latitude)

In [ ]:
destination_node

65344021

c. Calculate Optimal Route Considering Traffic

In [ ]:
# Modify edge weights based on traffic data
# This step requires processing traffic_data to adjust 'travel_time' or 'speed'

# Example: If traffic is heavy, increase travel time on affected edges
for edge in G.edges(data=True):
    if 'traffic_condition' in edge[2]:
        if edge[2]['traffic_condition'] == 'heavy':
            G[edge[0]][edge[1]][edge[2]['key']]['travel_time'] *= 1.5  # Increase travel time by 50%

# Compute the shortest path based on travel_time
route = nx.shortest_path(G, origin_node, destination_node, weight='travel_time')

## Charging Optimization

a. Determine EV's Range and Charging Needs

In [ ]:
# EV Specifications
battery_capacity_kWh = 75  # Example: 75 kWh
current_charge_kWh = 50
energy_consumption_kWh_per_mile = 0.3  # Example consumption rate

# Calculate remaining range
remaining_range = current_charge_kWh / energy_consumption_kWh_per_mile  # in miles

In [ ]:
remaining_range

166.66666666666669

b. Identify Charging Stations Along the Route

In [ ]:
# Convert route nodes to geographic coordinates
route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]

# Function to find charging stations within a certain distance from the route
from shapely.geometry import Point, LineString

route_line = LineString([(x[1], x[0]) for x in route_coords])  # Note: shapely uses (x, y)

def find_nearby_stations(charging_df, route_line, buffer=0.09):
    """Find charging stations within buffer degrees of the route."""
    nearby = charging_df[
        charging_df.apply(
            lambda row: route_line.distance(Point(row['Longitude'], row['Latitude'])) > 0, axis=1
        )
    ]
    return nearby

nearby_stations = find_nearby_stations(charging_df, route_line)

In [ ]:
nearby_stations

,Latitude,Longitude,Station_Name,ConnectionTypeID,Amps,Voltage,PowerKW,Quantity,Comments
0,35.189918,-106.636386,Melloy Dodge,1,80.0,208.0,19.2,2,Near Building
1,35.189918,-106.636386,Melloy Dodge,32,200.0,480.0,125.0,2,Under solar canopy
2,35.189918,-106.636386,Melloy Dodge,2,200.0,480.0,125.0,2,Under solar canopy
3,35.189918,-106.636386,Melloy Dodge,32,400.0,480.0,180.0,2,Under solar canopy
4,43.038698,-108.379586,Fremont Chevrolet Buick GMC,32,NaN,NaN,60.0,2,NaN
...,...,...,...,...,...,...,...,...,...
1241,39.985070,-105.229440,Iconergy Boulder,1,16.0,230.0,3.7,4,kW power is an estimate based on the connectio...
1242,39.972543,-76.671386,AAA MID STATES AAA MID STATES1,1,16.0,230.0,3.7,2,kW power is an estimate based on the connectio...
1243,39.967285,-83.006446,LAZ COLUMBUS OH MARCONI BLVD,1,16.0,230.0,3.7,2,kW power is an estimate based on the connectio...
1244,39.966927,-75.136540,PHILLY VM PHILLY VM BASE,1,16.0,230.0,3.7,1,kW power is an estimate based on the connectio...


c. Optimize Charging Stops

In [ ]:
# Define the optimization problem
prob = LpProblem("EV_Charging_Optimization", LpMinimize)

# Variables: Whether to stop at a charging station or not
charging_stops = LpVariable.dicts("Stop", nearby_stations.index, 0, 1, LpBinary)

# Objective: Minimize total charging time or cost
# For simplicity, minimize the number of stops
prob += lpSum([charging_stops[i] for i in nearby_stations.index])

# Constraints:
# Ensure that the EV does not run out of battery between charging stops
# This requires calculating distances between charging stations and ensuring they are within the EV's range

# Example constraint (simplified):
# Assuming stations are ordered along the route
max_range_miles = remaining_range
for i in range(len(nearby_stations) - 1):
    distance = ...  # Calculate distance between station i and i+1
    prob += charging_stops[i] + charging_stops[i+1] >= 1  # At least one stop within range

# Solve the problem
prob.solve()

# Extract selected charging stops
# selected_stops = nearby_stations[charging_stops[i].varValue == 1 for i in nearby_stations.index]
selected_stops = [station for i, station in nearby_stations.iterrows() if charging_stops[i].varValue == 1]


In [ ]:
selected_stops

[Latitude                     35.189918
 Longitude                  -106.636386
 Station_Name              Melloy Dodge
 ConnectionTypeID                    32
 Amps                             200.0
 Voltage                          480.0
 PowerKW                          125.0
 Quantity                             2
 Comments            Under solar canopy
 Name: 1, dtype: object,
 Latitude                     35.189918
 Longitude                  -106.636386
 Station_Name              Melloy Dodge
 ConnectionTypeID                    32
 Amps                             400.0
 Voltage                          480.0
 PowerKW                          180.0
 Quantity                             2
 Comments            Under solar canopy
 Name: 3, dtype: object,
 Latitude                          38.886198
 Longitude                        -77.431604
 Station_Name        G&C EV Charging Station
 ConnectionTypeID                         27
 Amps                                   28.0
 Volt

Next step is that modeling the problem considering exact distances, remaining battery after each segment, charging durations, and possibly charging costs.

## Visualization

a. Plot the Route and Charging Stations

In [ ]:
selected_stops_df = pd.DataFrame(selected_stops)

In [ ]:
# Initialize map centered at the origin
m = folium.Map(location=[origin.latitude, origin.longitude], zoom_start=13)

# Plot the route
route_latlng = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
folium.PolyLine(route_latlng, color="blue", weight=5, opacity=0.8).add_to(m)

# Plot charging stations
for idx, row in selected_stops_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Station_Name'],
        icon=folium.Icon(color='green', icon='bolt', prefix='fa')
    ).add_to(m)

# Display the map
m.save("/content/drive/MyDrive/Renewable_Energy/ev_route.html")

b. Display the Map

Open the generated ev_route.html in a web browser to interactively view the optimized route and charging stops.